In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def extract_all_distances(directory):
    all_distances = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path)
            all_distances.extend(df['Distance'].tolist())
    return all_distances

def get_color_based_on_quartile(distance, quartiles):
    """ Determine the color of the bar based on the quartile range. """
    if distance <= quartiles[0]:
        return 'green'  # Very close
    elif distance <= quartiles[1]:
        return 'blue'   # Quite close
    elif distance <= quartiles[2]:
        return 'orange' # Close
    else:
        return 'red'    # Not that close

def create_mosaic_of_charts(directory, quartiles):
    files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    num_files = len(files)
    cols = 3  # Adjust columns based on preference
    rows = (num_files + cols - 1) // cols

    sns.set(style="whitegrid")
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 10, rows * 6))

    color_legend = {
        'green': 'Very close',
        'blue': 'Quite close',
        'orange': 'Close',
        'red': 'Not that close'
    }

    y_lims = (0, 4)  # Set common y-axis limits

    for ax, file in zip(axes.flat, files):
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        character_names = df['name']
        distances = df['Distance']
        colors = [get_color_based_on_quartile(dist, quartiles) for dist in distances]

        ax.bar(character_names, distances, color=colors)
        ax.set_title(f"Distances from {file.replace('.csv', '').split('closest_to_')[1]} to Others", fontsize=10)
        ax.set_xlabel('Characters', fontsize=8)
        ax.set_ylabel('Distance', fontsize=8)
        ax.tick_params(axis='x', labelrotation=45, labelsize=8)
        ax.tick_params(axis='y', labelsize=8)
        ax.set_ylim(y_lims)  # Apply the uniform y-axis limits here
        ax.legend([plt.Rectangle((0,0),1,1, color=color) for color in color_legend], [color_legend[color] for color in color_legend.keys()], title="Quartile Range", fontsize=8)

        # Save each individual plot
        individual_plot_path = os.path.join(directory, f"chart_{file.replace('.csv', '')}.png")
        fig_individual = plt.figure(figsize=(10, 6))
        plt.bar(character_names, distances, color=colors)
        plt.title(f"Distances from {file.replace('.csv', '').split('closest_to_')[1]} to Others", fontsize=10)
        plt.xlabel('Characters', fontsize=8)
        plt.ylabel('Distance', fontsize=8)
        plt.ylim(y_lims)
        plt.xticks(rotation=45, fontsize=8)
        plt.legend([plt.Rectangle((0,0),1,1, color=color) for color in color_legend], [color_legend[color] for color in color_legend.keys()], title="Quartile Range", fontsize=8)
        plt.tight_layout()
        plt.savefig(individual_plot_path)
        plt.close(fig_individual)

    # Hide any unused axes
    for ax in axes.flat[len(files):]:
        ax.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(directory, 'all_charts_mosaic.png'))
    plt.close()

def main(directory):
    all_distances = extract_all_distances(directory)
    quartiles = np.percentile(all_distances, [25, 50, 75])
    create_mosaic_of_charts(directory, quartiles)

directory_path = '.'  # Replace with your directory path
main(directory_path)
